## **Librerías**

In [2]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns

In [3]:
pd.options.display.max_columns = False

In [4]:
from datetime import datetime, timedelta
import winsound as cst

In [5]:
import sys
sys.path.append('./')
sys.path.append('../')

In [6]:
import Utils.consultas as cst 

In [6]:
#query de migra 
query=f'''
SELECT 16 as campania, A.msisdn as numero,CUST.CUST_ID,
CASE 
    WHEN ROWNUM <=90000 THEN 'G1'
    WHEN ROWNUM BETWEEN 90001 AND 180000 THEN 'G2'
    WHEN ROWNUM BETWEEN 180001 AND 270000 THEN 'G3'
    WHEN ROWNUM >270000 THEN 'G4'
END AS GRUPO
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
LEFT JOIN DWH_BODEGA_WOM.DIM_CLIENTES CLI ON A.CLIENTE_DK = CLI.CLIENTE_DK
LEFT JOIN DWH_REPLICA_WOM.HIT_CUST_D CUST ON CLI.CLIENTE_CODIGO_NK = CUST.CUST_ID
WHERE A.PERIODO_PROCESO_CODIGO=202312
AND A.SERVICIO='Prepaid'
AND A.MOVIMIENTO_NOMBRE='PERMANECE'
AND A.MSISDN IN (
SELECT MSISDN FROM CCERON_MBI_EXT.TBL_POTENCIALES_MIG_UPS A
WHERE A.CATEGORIA = 'MIGRAS'
)
AND CUST.CUST_ID IS NOT NULL 
'''
prueba=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [8]:
prueba

,CAMPANIA,NUMERO,CUST_ID,GRUPO
0,16,3238048627,220365238,G1
1,16,3025712407,220365255,G1
2,16,3028428523,220370702,G1
3,16,3244686175,220370703,G1
4,16,3027440897,220370751,G1
...,...,...,...,...
221864,16,3238446008,118432856,G3
221865,16,3238454489,118433351,G3
221866,16,3238454491,118433353,G3
221867,16,3138092527,118433376,G3


In [7]:
prueba.to_csv('migra_popup.csv')

In [6]:
#query de amparadas
query=f'''
 WITH 
    ENDING_LINEAS AS (
        SELECT 
            CLIENTE_DK, 
            MOVIMIENTO_NOMBRE, 
            SUBSCRIBER_ID, 
            FECHA_ALTA, 
            A.PLAN_COMERCIAL_DK, 
            B.PLAN_CODIGO_NK, 
            MSISDN,
            SERVICIO,
            COUNT(*) OVER (PARTITION BY CLIENTE_DK) N_LINEAS
        FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
        LEFT JOIN DWH_BODEGA_WOM.DIM_PLANES B ON A.PLAN_COMERCIAL_DK=B.PLAN_COMERCIAL_DK
            WHERE PERIODO_PROCESO_CODIGO = 202401
            --AND SERVICIO = 'Postpaid'
            AND MOVIMIENTO_NOMBRE <> 'DESCONEXION'
            AND PLAN_TIPO_COMERCIAL = 'Mobile'
            AND CLIENTE_DK NOT IN (-2)
    ), 
    
    SEGUNDA_LINEA AS (
        SELECT 
            SUBSCRIBER_ID, 
            PERIODO_OFERTA AS PERIODO_OFERTA_INI
        FROM (
            SELECT 
                SUBS_A AS SUBSCRIBER_ID, 
                TIPO,  
                TO_CHAR(FECHA_OFERTA, 'YYYYMM') AS PERIODO_OFERTA, 
                SUBS_CON_DESCUENTO, 
                ACTIVO
            FROM DWH_STAGE_WOM.STG_PROGRA_DESCUENTO_2X1 
            
            UNION ALL 
            
            SELECT 
                SUBS_B AS SUBSCRIBER_ID, 
                TIPO,  
                TO_CHAR(FECHA_OFERTA, 'YYYYMM') AS PERIODO_OFERTA, 
                SUBS_CON_DESCUENTO, 
                ACTIVO
            FROM DWH_STAGE_WOM.STG_PROGRA_DESCUENTO_2X1 
            ) 
        GROUP BY SUBSCRIBER_ID, PERIODO_OFERTA
        
        UNION ALL 
        
        SELECT 
            SUBSCRIBER_ID, 
            PERIODO_OFERTA_INI
        FROM PRUIZ_ANA.ANALITICA_2X1
    ), 
    
    BASE_AMPARADA AS (
        SELECT 
            LN.*, 
            CASE
                WHEN TO_NUMBER(PLAN_CODIGO_NK) IN (34979, 34981, 34971, 34973, 34969, 34980, 34982, 34972, 34974, 34970, 31961, 35082, 32289, 32027, 35083, 35086, 31963, 31967) THEN 'Presente Campa�a'
                ELSE 'Ausente Campa�a'
            END AS TIPO_AMP
        FROM ENDING_LINEAS LN 
        WHERE NOT EXISTS (SELECT 1 FROM DWH_CONSULTA_WOM.ANA_AMPARADAS AMP WHERE LN.SUBSCRIBER_ID=AMP.SUBSCRIBER_ID)
            AND NOT EXISTS (SELECT 1 FROM SEGUNDA_LINEA SL WHERE TO_CHAR(LN.SUBSCRIBER_ID)=SL.SUBSCRIBER_ID)
            AND N_LINEAS >= 1
            AND N_LINEAS < 5
            AND TO_NUMBER(PLAN_CODIGO_NK) IN (34979, 34981, 34971, 34973, 34969, 34980, 34982, 34972, 34974, 34970, 31961, 35082, 32289, 32027, 35083, 35086, 31963, 31967)
    ) 
select  15 as campania, A.msisdn as numero,CUST.CUST_ID from (
SELECT 
    EN.*, 
    AMP.TIPO_AMP, 
    UPS.TIPO_CAMPANA AS TIPO_UPSELL, 
    MIG.TIPO_CAMPANA AS TIPO_MIGRA
FROM ENDING_LINEAS EN
LEFT JOIN BASE_AMPARADA AMP ON EN.SUBSCRIBER_ID=AMP.SUBSCRIBER_ID
LEFT JOIN ( 
    SELECT MSISDN, CATEGORIA TIPO_CAMPANA
    FROM CCERON_MBI_EXT.TBL_POTENCIALES_MIG_UPS
    WHERE CATEGORIA = 'UPSELLING' --AND PERIODO = 202312
    ) UPS ON EN.MSISDN=UPS.MSISDN
LEFT JOIN ( 
        SELECT 
            MSISDN, 
            CATEGORIA TIPO_CAMPANA
        FROM CCERON_MBI_EXT.TBL_POTENCIALES_MIG_UPS
        WHERE CATEGORIA = 'MIGRAS' --AND PERIODO = 202312
    ) MIG ON EN.MSISDN=MIG.MSISDN	
    ) A
    LEFT JOIN DWH_BODEGA_WOM.DIM_CLIENTES CLI ON A.CLIENTE_DK = CLI.CLIENTE_DK
    LEFT JOIN DWH_REPLICA_WOM.HIT_CUST_D CUST ON CLI.CLIENTE_CODIGO_NK = CUST.CUST_ID
    where tipo_amp is not null
'''
prueba=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
prueba

,CAMPANIA,NUMERO,CUST_ID
0,15,3026661642,223714226
1,15,3025877060,227242479
2,15,3216926156,229920629
3,15,3135028206,18246290
4,15,3223291420,1225435977
...,...,...,...
964641,15,3026691698,222978153
964642,15,3238380209,229862532
964643,15,3144542446,18477230
964644,15,3158477392,12939502


In [9]:
prueba2=prueba

In [10]:

grupo_size = 90000
prueba2['GRUPO'] = (prueba2.index // grupo_size) + 1


print(prueba2)


        CAMPANIA      NUMERO     CUST_ID  GRUPO
0             15  3026661642   223714226      1
1             15  3025877060   227242479      1
2             15  3216926156   229920629      1
3             15  3135028206    18246290      1
4             15  3223291420  1225435977      1
...          ...         ...         ...    ...
964641        15  3026691698   222978153     11
964642        15  3238380209   229862532     11
964643        15  3144542446    18477230     11
964644        15  3158477392    12939502     11
964645        15  3027904972    17895492     11

[964646 rows x 4 columns]


In [11]:
prueba2.to_csv('amparada_popup.csv')

In [7]:
query=f'''
WITH 
    ENDING_LINEAS AS (
        SELECT 
            CLIENTE_DK, 
            MOVIMIENTO_NOMBRE, 
            SUBSCRIBER_ID, 
            FECHA_ALTA, 
            A.PLAN_COMERCIAL_DK, 
            B.PLAN_CODIGO_NK, 
            MSISDN,
            SERVICIO,
            COUNT(*) OVER (PARTITION BY CLIENTE_DK) N_LINEAS
        FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
        LEFT JOIN DWH_BODEGA_WOM.DIM_PLANES B ON A.PLAN_COMERCIAL_DK=B.PLAN_COMERCIAL_DK
            WHERE PERIODO_PROCESO_CODIGO = 202401
            --AND SERVICIO = 'Postpaid'
            AND MOVIMIENTO_NOMBRE <> 'DESCONEXION'
            AND PLAN_TIPO_COMERCIAL = 'Mobile'
            AND CLIENTE_DK NOT IN (-2)
    ), 
    
    SEGUNDA_LINEA AS (
        SELECT 
            SUBSCRIBER_ID, 
            PERIODO_OFERTA AS PERIODO_OFERTA_INI
        FROM (
            SELECT 
                SUBS_A AS SUBSCRIBER_ID, 
                TIPO,  
                TO_CHAR(FECHA_OFERTA, 'YYYYMM') AS PERIODO_OFERTA, 
                SUBS_CON_DESCUENTO, 
                ACTIVO
            FROM DWH_STAGE_WOM.STG_PROGRA_DESCUENTO_2X1 
            
            UNION ALL 
            
            SELECT 
                SUBS_B AS SUBSCRIBER_ID, 
                TIPO,  
                TO_CHAR(FECHA_OFERTA, 'YYYYMM') AS PERIODO_OFERTA, 
                SUBS_CON_DESCUENTO, 
                ACTIVO
            FROM DWH_STAGE_WOM.STG_PROGRA_DESCUENTO_2X1 
            ) 
        GROUP BY SUBSCRIBER_ID, PERIODO_OFERTA
        
        UNION ALL 
        
        SELECT 
            SUBSCRIBER_ID, 
            PERIODO_OFERTA_INI
        FROM PRUIZ_ANA.ANALITICA_2X1
    ), 
    
    BASE_AMPARADA AS (
        SELECT 
            LN.*, 
            CASE
                WHEN TO_NUMBER(PLAN_CODIGO_NK) IN (34979, 34981, 34971, 34973, 34969, 34980, 34982, 34972, 34974, 34970, 31961, 35082, 32289, 32027, 35083, 35086, 31963, 31967) THEN 'Presente Campa�a'
                ELSE 'Ausente Campa�a'
            END AS TIPO_AMP
        FROM ENDING_LINEAS LN 
        WHERE NOT EXISTS (SELECT 1 FROM DWH_CONSULTA_WOM.ANA_AMPARADAS AMP WHERE LN.SUBSCRIBER_ID=AMP.SUBSCRIBER_ID)
            AND NOT EXISTS (SELECT 1 FROM SEGUNDA_LINEA SL WHERE TO_CHAR(LN.SUBSCRIBER_ID)=SL.SUBSCRIBER_ID)
            AND N_LINEAS >= 1
            AND N_LINEAS < 5
            AND TO_NUMBER(PLAN_CODIGO_NK) IN (34979, 34981, 34971, 34973, 34969, 34980, 34982, 34972, 34974, 34970, 31961, 35082, 32289, 32027, 35083, 35086, 31963, 31967)
    ) 
select  17 as campania, A.msisdn as numero,CUST.CUST_ID from (
SELECT 
    EN.*, 
    AMP.TIPO_AMP, 
    UPS.TIPO_CAMPANA AS TIPO_UPSELL, 
    MIG.TIPO_CAMPANA AS TIPO_MIGRA
FROM ENDING_LINEAS EN
LEFT JOIN BASE_AMPARADA AMP ON EN.SUBSCRIBER_ID=AMP.SUBSCRIBER_ID
LEFT JOIN ( 
    SELECT MSISDN, CATEGORIA TIPO_CAMPANA
    FROM CCERON_MBI_EXT.TBL_POTENCIALES_MIG_UPS
    WHERE CATEGORIA = 'UPSELLING' --AND PERIODO = 202312
    ) UPS ON EN.MSISDN=UPS.MSISDN
LEFT JOIN ( 
        SELECT 
            MSISDN, 
            CATEGORIA TIPO_CAMPANA
        FROM CCERON_MBI_EXT.TBL_POTENCIALES_MIG_UPS
        WHERE CATEGORIA = 'MIGRAS' --AND PERIODO = 202312
    ) MIG ON EN.MSISDN=MIG.MSISDN	
    ) A
    LEFT JOIN DWH_BODEGA_WOM.DIM_CLIENTES CLI ON A.CLIENTE_DK = CLI.CLIENTE_DK
    LEFT JOIN DWH_REPLICA_WOM.HIT_CUST_D CUST ON CLI.CLIENTE_CODIGO_NK = CUST.CUST_ID
    where tipo_upsell is not null
'''
prueba=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [8]:
prueba

,CAMPANIA,NUMERO,CUST_ID
0,17,3146492533,15542613
1,17,3123830854,13462271
2,17,3162948113,14412831
3,17,3007097811,17687493
4,17,3197110562,226401807
...,...,...,...
363059,17,3213527110,14073536
363060,17,3028639807,224817297
363061,17,3125910766,223397135
363062,17,3012566810,16685672


In [9]:

grupo_size = 90000
prueba['GRUPO'] = (prueba.index // grupo_size) + 1


print(prueba)

        CAMPANIA      NUMERO    CUST_ID  GRUPO
0             17  3146492533   15542613      1
1             17  3123830854   13462271      1
2             17  3162948113   14412831      1
3             17  3007097811   17687493      1
4             17  3197110562  226401807      1
...          ...         ...        ...    ...
363059        17  3213527110   14073536      5
363060        17  3028639807  224817297      5
363061        17  3125910766  223397135      5
363062        17  3012566810   16685672      5
363063        17  3157194119   15526301      5

[363064 rows x 4 columns]


In [10]:
prueba.to_csv('upselling_popup.csv')

## **Variables Globales**

In [6]:
dia = int(str(datetime.now()).replace('-', '')[:8])
mes = int(str(datetime.now()).replace('-', '')[:6])

## **Datos**

## **Limpieza**

## **Analísis y Exploración**